# Setup

In [1]:
import pandas as pd
import numpy as np
import os

# To serialise models
from sklearn.externals import joblib

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import seaborn as sns

# plt.rcParams['axes.labelsize'] = 12
# plt.rcParams['xtick.labelsize'] = 9
# plt.rcParams['ytick.labelsize'] = 9
# plt.rcParams['axes.titlesize'] = 18
# plt.rcParams['figure.dpi'] = 400


plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 18

plt.style.use('fivethirtyeight')


# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=400):
    if not os.path.isdir(IMAGES_PATH):
        os.makedirs(IMAGES_PATH)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


PROCESSED_PATH = os.path.join(PROJECT_ROOT_DIR, "processed")
def save_processed(df, filename, extension="csv"):
    if not os.path.isdir(PROCESSED_PATH):
        os.makedirs(PROCESSED_PATH)
    path = os.path.join(PROCESSED_PATH, filename + "." + extension)
    print("Saving processed dataset", filename)
    df.to_csv(path, index=False)
    
# if less than R1K, show as R800 (for eg). If btw R1000 and R999,999, show as R1K or R999K. If R1,000,000 or greater, show as R1M
def currency_formatter(x, pos):
    if x >= 1000000:
        return 'R%1.1fM' % (x * 1e-6)
    elif x < 1000:
        return 'R%1.1d' % (x)
    else:
        return 'R%1.1dK' % (x * 1e-3) 
    
axes_formatter = FuncFormatter(currency_formatter) 

# Download & Process Zomato Data

In [2]:
API_KEY = "0ce0b2e48571f88facd08f8efd3569aa"
ENTITY_TYPE = "city"
SORT_BY = "rating"
SORT_ORDER = ["desc", "asc"]
RAW_FOLDER = "raw/"
PROCESSED_FOLDER = "processed/"

In [3]:
def load_data(base_path, file, sub_dir=None, ext="csv", encoding=None):
    filename = file + "." + ext
    if sub_dir is not None:
        csv_path = os.path.join(base_path, sub_dir, filename)
    else:
        csv_path = os.path.join(base_path, filename)
    return pd.read_csv(csv_path)

In [4]:
city_ids = load_data(RAW_FOLDER, "zomato_city_ids")
city_ids.head()

,Country,City,Zomato ID
0,South Africa,Cape Town,64


In [5]:
import requests
import json

def get_categories():
    # Get all zomato categories.
    # Categories are global and not unique to a specific country.
    headers = {"user-key": API_KEY}
    response=requests.get("https://developers.zomato.com/api/v2.1/categories", headers=headers)
    json_data = response.json()
    
    if not os.path.isdir(RAW_FOLDER):
         os.makedirs(RAW_FOLDER)
    file_name = "categories.json"
    file_path = os.path.join(RAW_FOLDER, file_name)
    
    with open(file_path, 'w') as outfile:
        json.dump(json_data, outfile)
        
    # return all the category id's for restaurant search function.  
    cat_ids = [category["categories"]["id"] for category in json_data["categories"]]
    return cat_ids
        
def get_establishment_type_for_city(city_id):
    # Finding all establishment types in city_id and writing to file
    headers = {"user-key": API_KEY}
    params = {"city_id": city_id}
    response=requests.get("https://developers.zomato.com/api/v2.1/establishments", 
                          headers=headers, params=params)
    json_data = response.json()
       
    if not os.path.isdir(RAW_FOLDER):
         os.makedirs(RAW_FOLDER)    
    file_name = "establishment_types_" + str(city_id) + "." + "json"
    file_path = os.path.join(RAW_FOLDER, file_name)
    
    with open(file_path, 'w') as outfile:
        json.dump(json_data, outfile)
        
    # return all the establishment type id's for restaurant search function.  
    establishment_ids = [establishment["establishment"]["id"] for establishment in json_data["establishments"]]
    return establishment_ids
        
def get_cuisine_type_for_city(city_id):
    # Finding all cuisine types in Cape Town and writing to file
    headers = {"user-key": API_KEY}
    params = {"city_id": city_id}
    response=requests.get("https://developers.zomato.com/api/v2.1/cuisines", 
                          headers=headers, params=params)
    json_data = response.json()
    
    if not os.path.isdir(RAW_FOLDER):
         os.makedirs(RAW_FOLDER)    
    file_name = "cuisine_types_" + str(city_id) + "." + "json"
    file_path = os.path.join(RAW_FOLDER, file_name)
    
    with open(file_path, 'w') as outfile:
        json.dump(json_data, outfile)
        
    # return all the cuisine type id's for restaurant search function.
    cuisine_ids = [cuisine["cuisine"]["cuisine_id"] for cuisine in json_data["cuisines"]]
    return cuisine_ids

In [6]:
def download_with_criteria(headers, city_id, file_path, iterable_list, iterable_name, sort_order, is_item_list=False):
    json_dump = []
    
    # if API is not expecting a list for iterable_name, then run through each value as as search criteria
    if not is_item_list:        
        for item in iterable_list:
            start = 0
            results_shown = 20
            while results_shown != 0:                
                params = {"entity_id": city_id, "entity_type": ENTITY_TYPE, "start": start, 
                          "count": 20, iterable_name: item,"sort": SORT_BY, 
                          "order": sort_order}
                response=requests.get("https://developers.zomato.com/api/v2.1/search", 
                                  headers=headers, params=params)

                json_data = response.json()
                results_shown = int(json_data.get("results_shown", 0))

                if results_shown == 0:
                    break

                # appending each dictionary to a list so that json.load() can process 
                # multiple dictionaries
                json_dump.append(json.dumps(json_data))     

                start += 20
    
    # otherwise pass the full list to iterable_name, such as cuisine's. 
    # note: initially it was attempted to run through the above loop for every cuisine type, but allowed API calls 
    # where exceeded for a single city download attempt. 
    else:
        start = 0
        results_shown = 20
        while results_shown != 0:            
            params = {"entity_id": city_id, "entity_type": ENTITY_TYPE, "start": start, 
                      "count": 20, iterable_name: iterable_list,"sort": SORT_BY, 
                      "order": sort_order}
            response=requests.get("https://developers.zomato.com/api/v2.1/search", 
                              headers=headers, params=params)

            json_data = response.json()
            results_shown = int(json_data.get("results_shown", 0))

            if results_shown == 0:
                break

            # appending each dictionary to a list so that json.load() can process 
            # multiple dictionaries
            json_dump.append(json.dumps(json_data))     

            start += 20
        
            
    
    # each dictionary must exist as a list object for json.load to read it correctly    
    with open(file_path, 'w') as outfile:
        outfile.write("[")
        count = 0
        for item in json_dump:
            outfile.write(item)
            if count < len(json_dump)-1:
                outfile.write(",")
            count += 1
        outfile.write("]")

In [7]:
def get_restaurants(city_id, establishment_types):
     # Find 100 top rated restaurants of each establishment type, category and cuisine type per city and write to file
    headers = {"user-key": API_KEY}
    
    if not os.path.isdir(RAW_FOLDER):
         os.makedirs(RAW_FOLDER)
    
    search_criteria = ["Category", "Establishment Type", "Cuisine Type"]
    city_name = city_ids[city_ids["Zomato ID"] == city_id]["City"].values[0]

    # Download restaurant for city by establishment type
    print("Downloading restaurants for", city_name, "by", search_criteria[1])
    
    for sort_order in SORT_ORDER:
        file_name = "restaurants_by_establishment_type_" + str(city_id) + "_" + sort_order + "." + "json"
        file_path = os.path.join(RAW_FOLDER, file_name)
        download_with_criteria(headers, city_id, file_path, establishment_types, "establishment_type", 
                               sort_order, is_item_list=False)
    print("Done")

In [8]:
for city_id in city_ids["Zomato ID"]:
    establishments = get_establishment_type_for_city(city_id)
    get_restaurants(city_id, establishments)

ConnectionError: HTTPSConnectionPool(host='developers.zomato.com', port=443): Max retries exceeded with url: /api/v2.1/establishments?city_id=64 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a19291160>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
# For each city, we need to combine the restaurant data from the categories-based, establishment type-based and cuisine-based json files. 

def build_csv(city_id, source_type):
    restaurants_list = []
    for sort_order in SORT_ORDER:
        file_name = "restaurants_by_" + source_type + "_" + str(city_id) + "_" + sort_order + ".json"
        source_path = os.path.join(RAW_FOLDER, file_name)

        db = json.load(open(source_path))

        restaurant_cols = ["name", 
                           "cuisines",
                           "aggregate_rating", 
                           "rating_text", 
                           "votes",
                           "currency", 
                           "average_cost_for_two", 
                           "price_range",                    
                           "locality",
                           "locality_verbose",
                           "city", 
                           "zipcode",
                           "country_id",
                           "latitude", 
                           "longitude",                                       
                           "has_online_delivery", 
                           "has_table_booking", 
                           ]

        restaurants = [restaurant_group["restaurant"] for count_group in db 
                           for restaurant_group in count_group["restaurants"]] 

        for restaurant in restaurants:
            restaurant["locality"] = restaurant["location"]["locality"]
            restaurant["locality_verbose"] = restaurant["location"]["locality_verbose"]
            restaurant["city"] = restaurant["location"]["city"]
            restaurant["latitude"] = restaurant["location"]["latitude"]
            restaurant["longitude"] = restaurant["location"]["longitude"]
            restaurant["zipcode"] = restaurant["location"]["zipcode"]
            restaurant["country_id"] = restaurant["location"]["country_id"]
            restaurant["aggregate_rating"] = restaurant["user_rating"]["aggregate_rating"]
            restaurant["votes"] = restaurant["user_rating"]["votes"]
            restaurant["rating_text"] = restaurant["user_rating"]["rating_text"]
            
        restaurants_list.append(restaurants)
        
    df1 = pd.DataFrame(restaurants_list[0], columns=restaurant_cols)
    df2 = pd.DataFrame(restaurants_list[1], columns=restaurant_cols)
    return pd.concat([df1, df2])
    

def process_restaurants_json(city_id):
    
    source_types = ["establishment_type"]
    dfs = []
    for source in source_types:
        dfs.append(build_csv(city_id, source))
    
    if not os.path.isdir(PROCESSED_FOLDER):
         os.makedirs(PROCESSED_FOLDER) 
    
    file_name = "restaurants_" + str(city_id) + ".csv"
    output_path = os.path.join(PROCESSED_FOLDER, file_name)

    data = pd.concat(dfs)
    data.to_csv(output_path, encoding='utf-8-sig', index=False)
    

In [ ]:
for city in city_ids["Zomato ID"]:
    process_restaurants_json(city)

# Data Preparation

In [91]:
data = load_data(PROCESSED_FOLDER, 'restaurants_64')
data.shape

(1970, 17)

In [92]:
data.head()

,name,cuisines,aggregate_rating,rating_text,votes,currency,average_cost_for_two,price_range,locality,locality_verbose,city,zipcode,country_id,latitude,longitude,has_online_delivery,has_table_booking
0,Moro Gelato,"Desserts, Ice Cream",4.8,Excellent,72,R,80,1,CBD,"CBD, Cape Town",Cape Town,NaN,189,-33.924108,18.417317,0,0
1,Unframed Ice Cream,Ice Cream,4.7,Excellent,104,R,40,1,Gardens,"Gardens, Cape Town",Cape Town,NaN,189,-33.928691,18.412097,0,0
2,The Velvet Cake Co.,"Bakery, Desserts",4.6,Excellent,90,R,165,2,CBD,"CBD, Cape Town",Cape Town,NaN,189,-33.919300,18.421512,0,0
3,The Creamery,"Desserts, Ice Cream",4.5,Excellent,389,R,110,2,Newlands,"Newlands, Cape Town",Cape Town,NaN,189,-33.970286,18.461950,0,0
4,My Sugar,"Cafe, Patisserie, Bakery, Desserts",4.5,Excellent,197,R,125,2,Sea Point,"Sea Point, Cape Town",Cape Town,NaN,189,-33.921692,18.382759,0,0


## Remove Duplicates

Since we downloaded the top 100 and bottom 100 restuarants of each category, there is likely to be some duplication as certain categories contain less than 200 restaurants meaning that the same establishment could appear near the bottom of the top 100 or the top of the bottom 100. There are many restaurant franchises in Cape Town, so we'll define a duplicate as a restaurant with the same name at the same locality.

In [93]:
data[data.duplicated(subset=["name", "locality"])]

,name,cuisines,aggregate_rating,rating_text,votes,currency,average_cost_for_two,price_range,locality,locality_verbose,city,zipcode,country_id,latitude,longitude,has_online_delivery,has_table_booking
87,N' Ice,"Ice Cream, Desserts",0.0,Not rated,1,R,70,1,Strand,"Strand, Cape Town",Cape Town,NaN,189,-34.114265,18.822527,0,0
156,Vida E Caffe,"Cafe, Desserts",3.3,Average,17,R,150,2,Gardens,"Gardens, Cape Town",Cape Town,NaN,189,-33.929000,18.411833,0,0
162,Kauai,"Healthy Food, Fast Food",3.3,Average,11,R,200,3,CBD,"CBD, Cape Town",Cape Town,NaN,189,-33.922833,18.420833,0,0
299,The Fat Cactus,"Mexican, Finger Food, Tex-Mex",4.1,Very Good,420,R,300,4,Gardens,"Gardens, Cape Town",Cape Town,NaN,189,-33.928833,18.410833,0,0
301,Rick's Cafe Americain,"Moroccan, Grill, Seafood, Burger, Finger Food",3.9,Good,273,R,320,4,Gardens,"Gardens, Cape Town",Cape Town,NaN,189,-33.928372,18.410481,0,0
302,Beerhouse on Long,"Grill, German",3.9,Good,305,R,300,4,CBD,"CBD, Cape Town",Cape Town,8001.0,189,-33.925512,18.415936,0,0
303,Bombay Bicycle Club,"Mediterranean, Steak, Finger Food",3.9,Good,279,R,450,4,Gardens,"Gardens, Cape Town",Cape Town,NaN,189,-33.936626,18.404738,0,0
500,Warwick Wine Estate,"South African, Mediterranean, Cafe",4.6,Excellent,104,R,440,4,Stellenbosch,"Stellenbosch, Cape Town",Cape Town,NaN,189,-33.840835,18.864158,0,0
502,My Sugar,"Cafe, Patisserie, Bakery, Desserts",4.5,Excellent,197,R,125,2,Sea Point,"Sea Point, Cape Town",Cape Town,NaN,189,-33.921692,18.382759,0,0
503,The Velvet Cake Co.,"Cafe, Desserts, Bakery",4.4,Very Good,130,R,160,2,Bellville,"Bellville, Cape Town",Cape Town,NaN,189,-33.868301,18.641180,0,0


In [94]:
data.duplicated(subset=["name", "locality"]).sum()

614

In [95]:
# duplicates are considerd those restaurants with the same name in the same location. Keep the first instance.
data = data.drop_duplicates(subset=["name", "locality"], keep='first')
data.shape

(1356, 17)

## Removing Unneccessary Columns

In [96]:
#removing currency, country, locality_verbose and zipcode
cols = ['name', 'cuisines', 'aggregate_rating', 'rating_text', 'votes',
       'average_cost_for_two', 'price_range', 'locality', 'city',
       'latitude', 'longitude', 'has_online_delivery', 'has_table_booking']
data = data[cols]
data.head()

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking
0,Moro Gelato,"Desserts, Ice Cream",4.8,Excellent,72,80,1,CBD,Cape Town,-33.924108,18.417317,0,0
1,Unframed Ice Cream,Ice Cream,4.7,Excellent,104,40,1,Gardens,Cape Town,-33.928691,18.412097,0,0
2,The Velvet Cake Co.,"Bakery, Desserts",4.6,Excellent,90,165,2,CBD,Cape Town,-33.919300,18.421512,0,0
3,The Creamery,"Desserts, Ice Cream",4.5,Excellent,389,110,2,Newlands,Cape Town,-33.970286,18.461950,0,0
4,My Sugar,"Cafe, Patisserie, Bakery, Desserts",4.5,Excellent,197,125,2,Sea Point,Cape Town,-33.921692,18.382759,0,0


## Fix Restaurants with Missing Geographic Info

We'll also identify restaurants that don't have their latitude and longitude specified as we'll be using these parameters during the analysis. Where possible, we'll fill in these values by looking up the restaurant info on Google Maps. If we can't find the info, we'll remove the restaurant from the analysis. 

In [97]:
data[(data["longitude"] == 0) & (data["latitude"] == 0)]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking
41,Jessy's Waffles,Desserts,3.2,Average,11,100,2,Gardens,Cape Town,0.0,0.0,0,0
91,DV Café,"Desserts, Cafe, Ice Cream",0.0,Not rated,0,50,1,"Spice Route Farm, Paarl",Cape Town,0.0,0.0,0,0
130,Vida E Caffe,"Cafe, Desserts",3.5,Good,17,150,2,Camps Bay,Cape Town,0.0,0.0,0,0
134,Lekker Vegan,Fast Food,3.4,Average,28,200,3,CBD,Cape Town,0.0,0.0,0,0
361,The Vic Bar,"American, Italian",3.0,Average,6,0,1,Gardens,Cape Town,0.0,0.0,0,0
641,Ben's On the Beach,"Grill, Burger",3.6,Good,58,320,4,Strand,Cape Town,0.0,0.0,0,0
774,La Bella Rosa Bistro & Coffee,"Cafe, Burger, Bakery",2.8,Average,4,160,2,Durbanville,Cape Town,0.0,0.0,0,0
826,The Common Room,Finger Food,0.0,Not rated,0,70,1,CBD,Cape Town,0.0,0.0,0,0
937,Jack Rabbit Mobile Kitchen,"Fast Food, Street Food, South African",0.0,Not rated,3,100,2,Durbanville,Cape Town,0.0,0.0,0,0
1431,Moyo,African,3.4,Average,86,561,4,Newlands,Cape Town,0.0,0.0,0,0


In [98]:
data.loc[41, "latitude"] = -33.9314912; data.loc[41, "longitude"] = 18.416589
data.loc[91, "latitude"] = -33.7643602; data.loc[91, "longitude"] = 18.9180991
data.loc[130, "latitude"] = -33.9511; data.loc[130, "longitude"] = 18.3763713
data.loc[134, "latitude"] = -33.9284282; data.loc[134, "longitude"] = 18.4215289
data.loc[361, "latitude"] = -33.9330936; data.loc[361, "longitude"] = 18.4058024
data.loc[641, "latitude"] = -34.1115314; data.loc[641, "longitude"] = 18.819894
data.loc[774, "latitude"] = -33.833121; data.loc[774, "longitude"] = 18.6438816
data.loc[826, "latitude"] = -33.926449; data.loc[826, "longitude"] = 18.4454532
data.loc[1431, "latitude"] = -33.988727; data.loc[1431, "longitude"] = 18.4313973
data.loc[1540, "latitude"] = -33.9036187; data.loc[1540, "longitude"] = 18.4183216
data.loc[1663, "latitude"] = -33.9358607; data.loc[1663, "longitude"] = 18.8564851

In [99]:
data[(data["longitude"] == 0) & (data["latitude"] == 0)]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking
937,Jack Rabbit Mobile Kitchen,"Fast Food, Street Food, South African",0.0,Not rated,3,100,2,Durbanville,Cape Town,0.0,0.0,0,0


The only restaurant we couldn't find location information for was Jack Rabbit Mobile Kitchen. This makes sense since it sounds like a food truck that doesn't have a fixed location. We'll remove it.

In [100]:
data = data[(data["longitude"] != 0) & (data["latitude"] != 0)]
data[(data["longitude"] == 0) & (data["latitude"] == 0)]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking


We know these are South African restaurants, so all their latitude values should be negative and all their longitude values positive, let's check this.

In [101]:
data[data["latitude"] > 0]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking
643,The 41,"Contemporary, Seafood, Mediterranean, Sushi",3.5,Good,29,400,4,Camps Bay,Cape Town,33.948688,18.376794,0,0
685,Pearl Mountain,Drinks Only,3.3,Average,7,50,1,"Pearl Mountain Wine Estate, Paarl",Cape Town,33.702186,18.995174,0,0
868,Groot Constantia Winery,Drinks Only,3.9,Good,52,90,1,"Groot Constantia Wine Estate, Constantia",Cape Town,34.026732,18.422542,0,0
1665,Anura Tasting Room,Drinks Only,3.1,Average,5,100,2,Anura,Cape Town,33.804110,18.884110,0,0
1841,The Trading Post Eatery,"Cafe, Deli",3.0,Average,5,200,3,Anura,Cape Town,33.811946,18.885821,0,0


In [102]:
f = lambda x: -x if x > 0 else x
data.latitude = data.latitude.apply(f)
data[data["latitude"] > 0]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking


In [103]:
data[data["longitude"] < 0]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,longitude,has_online_delivery,has_table_booking


## Add Dummy Columns for Each Cuisine Type

We'd like to have a true/false value for each cuisine type. This will make it easier to count restaurants by specific cuisine types later. Zomato piles all the cuisines together in a single comma seperated field, so we need to break this up first and then create dummies.

In [104]:
# Values in cuisine columns may have multiple values with ',' seperator
# Extract each possible cuisine type then ensure each dummy column is prefixed with "cuisine_"

dummies = data['cuisines'].str.get_dummies(sep=", ")
dummies.columns = [str(col) for col in dummies.columns]
data = pd.concat([data, dummies], axis=1)
print(data.shape)
data.sample(10)

(1355, 86)


,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Steak,Street Food,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese
485,Blanko,Italian,3.7,Good,25,500,4,"Alphen 5 Star Boutique Hotel, Constantia",Cape Town,-34.010804,...,0,0,0,0,0,0,0,0,0,0
592,Chardonnay Deli,"Deli, Bakery, Cafe",3.7,Good,58,180,2,Constantia,Cape Town,-34.025286,...,0,0,0,0,0,0,0,0,0,0
0,Moro Gelato,"Desserts, Ice Cream",4.8,Excellent,72,80,1,CBD,Cape Town,-33.924108,...,0,0,0,0,0,0,0,0,0,0
1894,The Polo Club Restaurant,"South African, Cafe",3.2,Average,9,380,4,Paarl,Cape Town,-33.804584,...,0,0,0,0,0,0,0,0,0,0
909,Terra Del Capo Antipasti,Italian,3.6,Good,15,190,2,Franschhoek,Cape Town,-33.871929,...,0,0,0,0,0,0,0,0,0,0
978,The Chef’s Bench,"Street Food, Burger",0.0,Not rated,0,120,2,CBD,Cape Town,-33.922941,...,0,1,0,0,0,0,0,0,0,0
922,Franschhoek Kitchen,African,3.5,Good,24,420,4,Franschhoek,Cape Town,-33.937681,...,0,0,0,0,0,0,0,0,0,0
102,Osumo,Healthy Food,3.9,Good,76,150,2,"Cavendish Square, Claremont",Cape Town,-33.980257,...,0,0,0,0,0,0,0,0,0,0
1523,Woolworths Cafe,Cafe,2.7,Average,17,230,3,"Blue Route Mall, Tokai",Cape Town,-34.063988,...,0,0,0,0,0,0,0,0,0,0
1416,The Dining Room,South African,3.2,Average,13,680,4,Woodstock,Cape Town,-33.928085,...,0,0,0,0,0,0,0,0,0,0


## Extract a Place of Interest from Locality and Store in New Column
Some restaurants are part of a place of interest such as a shopping mall or hotel. This is specified in the locality as a comma-seperated pair. We'll split these out into their own column and use "None" where it doesn't apply. This allows us to investigate rating trends at specific shopping malls, for example.

In [105]:
# Go through each row and extract a place value from the locality if one is specified
# Store the place and locality seperately in order to filter more precisely

places = []
localities = []
def split_locality(row):
    items = row["locality"].split(',')
    if len(items) > 1:
        places.append(items[0])
        localities.append(items[1].strip())
    else:
        places.append("None")
        localities.append(items[0].strip())
        
       
data.apply(split_locality, axis=1)
data["places"] = places
data["locality"] = localities

In [106]:
data[["locality", "places"]].sample(10)

,locality,places
1231,CBD,None
798,V & A Waterfront,Hotel Cape Grace
773,Brackenfell,None
82,Green Point,Cape Quarter Lifestyle Center
56,Ottery,None
463,Bellville,None
954,Bellvile,Willowbridge Lifestyle Centre
1919,Stellenbosch,None
583,Stellenbosch,None
27,Stellenbosch,None


## Fix Localities Incorrectly Entered Into Zomato
A number of localities were found with spelling mistakes leading to an incorrect representation of the data. While we could exclude these I would rather try fix them manually so as to keep as much data as possible for the analysis.

In [107]:
mapping = {
    "Dubanville": "Durbanville",
    "Contantia": "Constantia",
    "Tableview": "Table View",
    "Belville":"Bellville",
    "Belvile":"Bellville",
    "Bellvile":"Bellville",
    "Bay Area": "Kalk Bay",
    "V&A Waterfront":"V & A Waterfront",
    "The Woodstock Exchange. Woodstock": "Woodstock",
    "Franschoek":"Franschhoek",
    "Somersetwest":"Somerset West",
    "Kuilsriver":"Kuils River",
    "Greenpoint":"Green Point",
    "Ridgeback Wine Farm": "Ridgeback Wine Farm, Paarl",
    "Anura": "Anura, Stellenbosch",
    "Hannover": "Hanover Park",
    "NH The Lord Charles Hotel": "NH The Lord Charles Hotel, Somerset West",
    "Fairbridge Mall": "Fairbridge Mall, Brackenfell",
    "Blouberg": "Bloubergstrand",
    "Two Oceans Beach": "Two Oceans Beach, Mouille Point",
}

def correct_incorrect_localities(col):
    if col in mapping:
        return mapping.get(col, 'None')
    else:
        return col
    
# If no mapping provided, return x
f = lambda x: mapping.get(x, x)
data.locality = data.locality.map(f)

# split locality and places again after corrections.
places = []
localities = []

def split_locality(row):
    items = row["locality"].split(',')
    if len(items) > 1:
        places.append(items[0])
        localities.append(items[1].strip())
    else:
        places.append(row["places"])
        localities.append(row["locality"])
        
data.apply(split_locality, axis=1)
data["places"] = places
data["locality"] = localities

In [108]:
data[data.places == "NH The Lord Charles Hotel"]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Street Food,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places
1405,The Garden Terrace - NH The Lord Charles,"European, Contemporary, Italian",3.1,Average,4,460,4,Somerset West,Cape Town,-34.06657,...,0,0,0,0,0,0,0,0,0,NH The Lord Charles Hotel
1406,La Vigna - NH The Lord Charles,"Seafood, Burger, Beverages, Contemporary, Dess...",3.1,Average,6,450,4,Somerset West,Cape Town,-34.06657,...,0,0,0,0,0,0,0,0,0,NH The Lord Charles Hotel


In [109]:
idx = data[data.locality.str.contains('Kalk', regex=False)].index
data.loc[idx, "places"] = "None"

In [110]:
data[data.locality.str.contains('Kalk', regex=False)]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Street Food,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places
15,The Ice Cafe,Desserts,3.7,Good,41,45,1,Kalk Bay,Cape Town,-34.126000,...,0,0,0,0,0,0,0,0,0,None
108,Kalkys,"Seafood, Fish and Chips, Fast Food",3.8,Good,167,180,2,Kalk Bay,Cape Town,-34.129317,...,0,0,0,0,0,0,0,0,0,None
127,Lucky Fish & Chips,"Fast Food, Seafood",3.5,Good,24,150,2,Kalk Bay,Cape Town,-34.129317,...,0,0,0,0,0,0,0,0,0,None
424,Harbour House Restaurant,"Seafood, Sushi",4.1,Very Good,160,500,4,Kalk Bay,Cape Town,-34.129281,...,0,1,0,0,0,0,0,0,0,None
480,Live Bait,"Seafood, Sushi, European",3.7,Good,76,500,4,Kalk Bay,Cape Town,-34.132044,...,0,1,0,0,0,0,0,0,0,None
529,Olympia Cafe & Deli,"Cafe, Deli",4.0,Very Good,162,200,3,Kalk Bay,Cape Town,-34.127530,...,0,0,0,0,0,0,0,0,0,None
586,Lekker,"Cafe, South African, European",3.7,Good,75,250,3,Kalk Bay,Cape Town,-34.126930,...,0,0,0,0,0,0,0,0,0,None


## Build Web Scraper & Add Region Column to Visualise Larger Geographical Areas

In addition to the locality/suburb, we'd like to visualise larger regional trends as well. To do this, we'll scrape a list of Cape Town suburbs and their associated regions from https://en.wikipedia.org/wiki/List_of_Cape_Town_suburbs. 

We'll export the suburb/region match to its own data file, then for each restaurant in our dataset, we will match its locality to this list of suburbs and assign the fitting region to a new column in the main dataset.

In [111]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get('https://en.wikipedia.org/wiki/List_of_Cape_Town_suburbs')
content = response.content
parser = BeautifulSoup(content, 'html.parser')

#Finding all regions in th Wiki page - these have the .mw-headline class
regions_list = [region.text for region in parser.select(".mw-headline")]
suburbs_by_region = {}                    

# The .wikitable class is assigned to each table of suburbs per region    
regions = parser.select(".wikitable")  

count = 0
for region in regions:
    # Finding each suburb while skipping columns with postal codes
    # Some fields have additional info in (), so splitting on those and removing
    suburbs = [suburb.text.split('(')[0] for suburb in region.select("td")[::3]]
    
    for suburb in suburbs:
        suburbs_by_region[suburb] = regions_list[count]
    count += 1  
 
# We need to manually add entries which are known suburbs of the greater Cape Town area
df = pd.DataFrame(list(suburbs_by_region.items()), columns=['locality', 'region'])
df.to_csv("processed/cape-town-suburbs.csv", encoding='utf-8-sig', index=False)

In [112]:
suburbs = pd.read_csv("processed/cape-town-suburbs.csv")
suburbs.sample(10)

,locality,region
64,Observatory,Southern Suburbs
25,Kensington,Northern Suburbs
28,Loevenstein,Northern Suburbs
72,SouthField,Southern Suburbs
69,Rondebosch East,Southern Suburbs
15,Zonnebloem,City Bowl
115,Strandfontein,Cape Flats
98,Grassy Park,Cape Flats
12,Vredehoek,City Bowl
40,Fresnaye,Atlantic Seaboard


In [113]:
def match_locality(row):
    suburb = suburbs[suburbs['locality'] == row['locality']]
    if suburb.shape[0] == 1:
        return suburb.iloc[0]['region']
    else:
        return "None"

data["region"] = data.apply(match_locality, axis=1)
data.head(10)

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places,region
0,Moro Gelato,"Desserts, Ice Cream",4.8,Excellent,72,80,1,CBD,Cape Town,-33.924108,...,0,0,0,0,0,0,0,0,None,None
1,Unframed Ice Cream,Ice Cream,4.7,Excellent,104,40,1,Gardens,Cape Town,-33.928691,...,0,0,0,0,0,0,0,0,None,City Bowl
2,The Velvet Cake Co.,"Bakery, Desserts",4.6,Excellent,90,165,2,CBD,Cape Town,-33.919300,...,0,0,0,0,0,0,0,0,None,None
3,The Creamery,"Desserts, Ice Cream",4.5,Excellent,389,110,2,Newlands,Cape Town,-33.970286,...,0,0,0,0,0,0,0,0,None,Southern Suburbs
4,My Sugar,"Cafe, Patisserie, Bakery, Desserts",4.5,Excellent,197,125,2,Sea Point,Cape Town,-33.921692,...,0,0,0,0,0,0,0,0,None,Atlantic Seaboard
5,The Velvet Cake Co.,"Cafe, Desserts, Bakery",4.4,Very Good,130,160,2,Bellville,Cape Town,-33.868301,...,0,0,0,0,0,0,0,0,None,Northern Suburbs
6,The Creamery,"Desserts, Ice Cream",4.2,Very Good,110,110,2,Green Point,Cape Town,-33.904093,...,0,0,0,0,0,0,0,0,None,Atlantic Seaboard
7,Myog,Desserts,4.1,Very Good,87,100,2,Gardens,Cape Town,-33.931500,...,0,0,0,0,0,0,0,0,None,City Bowl
8,The Creamery,"Desserts, Ice Cream",4.1,Very Good,45,75,1,Claremont,Cape Town,-33.981278,...,0,0,0,0,0,0,0,0,None,Southern Suburbs
9,DV Artisan Chocolate,Desserts,3.9,Good,24,70,1,Paarl,Cape Town,-33.760864,...,0,0,0,0,0,0,0,0,Spice Route Farm,None


In [114]:
# We need to manually add suburbs that are part of the wider Cape Town region including Winelands etc.
# We could exclude these entries from the analysis but we'd then be excluding areas such as Stellenbosch and Franshoek which are know food mecca's in Cape Town.

mapping = {
    "CBD":"City Bowl",
    "Century City":"Northern Suburbs",
    "Paarl":"Winelands",
    "Stellenbosch":"Winelands",
    "V & A Waterfront":"Atlantic Seaboard",
    "Kalk Bay":"South Peninsula",
    "Franschhoek": "Winelands",
    "Riebeek Kasteel":"West Coast",
    "Gordons Bay":"Helderberg",
    "Hermanus":"Overberg",
    "Robertson":"Winelands",
    "Wellington":"Winelands",
    "Woodstock": "City Bowl",
    "Noordhoek":"South Peninsula",
    "Worcester":"Winelands",
    "Zonnebloem": "City Bowl",
    "Rylands":"Cape Flats",
    "Malmesbury": "West Coast",
    "Darling": "West Coast",
    "Sunningdale":"Northern Suburbs",
    "Cape Town International Airport":"Northern Suburbs",
    "Neelsie Student Centre": "Winelands",
    "Scarborough": "South Peninsula"
}


def map_locality_to_region(row):
    if row['region'] == 'None':
        return mapping.get(row['locality'], 'None')
    else:
        return row['region']
    

data["region"] = data.apply(map_locality_to_region, axis=1)
data[data["region"] == "None"]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places,region
464,Two Oceans Restaurant,Seafood,3.8,Good,54,570,4,Cape Point,Cape Town,-34.356368,...,0,0,0,0,0,0,0,0,None,None


In [115]:
data = data.drop(data.index[data["region"] == "None"].tolist())
data[data["region"] == "None"]

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places,region


In [116]:
data.sample(50)

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places,region
820,Cloud 9 Rooftop Bar and Venue,Italian,2.7,Average,12,200,3,Tamboerskloof,Cape Town,-33.930039,...,0,0,0,0,0,0,0,0,None,City Bowl
1186,Eagle Eye Spur,"Grill, Steak, Burger",2.0,Poor,27,330,4,Observatory,Cape Town,-33.942358,...,0,0,0,0,0,0,0,0,St Peters Square,Southern Suburbs
227,Cheyne's,"International, Asian",4.2,Very Good,103,605,4,Hout Bay,Cape Town,-34.044566,...,0,0,0,0,0,0,0,0,None,Atlantic Seaboard
1532,Frater House,Cafe,2.7,Average,5,20,1,CBD,Cape Town,-33.925234,...,0,0,0,0,0,0,0,0,None,City Bowl
1616,Mezani,Finger Food,2.8,Average,1,200,3,Observatory,Cape Town,-33.941001,...,0,0,0,0,0,0,0,0,None,Southern Suburbs
1285,Pakalolo,"Mexican, Burger, Finger Food, Grill",2.5,Average,40,280,3,Hout Bay,Cape Town,-34.045500,...,0,0,0,0,0,0,0,0,None,Atlantic Seaboard
158,Little Lighthouse Seafood,Seafood,3.2,Average,16,190,2,Bellville,Cape Town,-33.862153,...,0,0,0,0,0,0,0,0,None,Northern Suburbs
1551,Wimpy,"Cafe, South African, Burger, Fast Food",2.8,Average,6,200,3,Brackenfell,Cape Town,-33.883748,...,0,0,0,0,0,0,0,0,Fairbridge Mall,Northern Suburbs
4,My Sugar,"Cafe, Patisserie, Bakery, Desserts",4.5,Excellent,197,125,2,Sea Point,Cape Town,-33.921692,...,0,0,0,0,0,0,0,0,None,Atlantic Seaboard
370,Bok Lounge,"Pizza, Finger Food",2.9,Average,9,200,3,Brackenfell,Cape Town,-33.873615,...,0,0,0,0,0,0,0,0,None,Northern Suburbs


Let's dive deeper and see if we can spot cuisine trends within specific suburbs. We'll look for the top 10 and bottom 10 

## Add Column to Store The Count of Cuisines Types Offered

In [117]:
def get_cuisine_cols():
    cols = ["name", "aggregate_rating", "rating_text", "votes", "average_cost_for_two", "price_range", "cuisines", "cuisine_count", "places", "locality", "region", "city", "latitude", "longitude", "has_online_delivery", "has_table_booking", "top_bottom"]
    '''Returns the column names of all the cuisine type column, not including the cuisine_count column'''
    cuisine_cols = [col for col in data.columns.tolist() if col not in cols]
    return cuisine_cols   

def count_cuisines(row):
    #count = len([col for col in data.columns if 'cuisine_' in col if row[col] == True]) 
    count = len([col for col in data.columns.tolist() if col in get_cuisine_cols() if row[col] == True]) 
    return count

data["cuisine_count"] = data.apply(count_cuisines, axis=1)
data.sample(5)

,name,cuisines,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,locality,city,latitude,...,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese,places,region,cuisine_count
933,The Restaurant at Grande Provence,"Mediterranean, European",3.4,Average,24,700,4,Franschhoek,Cape Town,-33.904411,...,0,0,0,0,0,0,0,None,Winelands,2
1553,Vetkoek Paleis,"Cafe, South African",2.9,Average,7,120,2,Strand,Cape Town,-34.113503,...,0,0,0,0,0,0,0,None,Helderberg,2
152,Kauai,"Healthy Food, Fast Food",3.3,Average,20,200,3,Stellenbosch,Cape Town,-33.931611,...,0,0,0,0,0,0,0,De Wet Square,Winelands,2
6,The Creamery,"Desserts, Ice Cream",4.2,Very Good,110,110,2,Green Point,Cape Town,-33.904093,...,0,0,0,0,0,0,0,None,Atlantic Seaboard,2
540,Max Bagels,"Cafe, Bakery",4.0,Very Good,66,0,1,CBD,Cape Town,-33.922269,...,0,0,0,0,0,0,0,None,City Bowl,2


In [118]:
data[["cuisines", "cuisine_count"]].sample(10)

,cuisines,cuisine_count
1627,Thai,1
255,European,1
1644,Drinks Only,1
912,Cafe,1
1899,"Cafe, Sushi, Pizza, Burger",4
29,"Cafe, Desserts",2
95,Ice Cream,1
688,Drinks Only,1
1504,Cafe,1
1387,"European, Contemporary",2


## Reorder Columns & Export Prepared CSV

In [119]:
col_order = ['name', 'aggregate_rating', 'rating_text', 'votes',
       'average_cost_for_two', 'price_range', 'cuisines', 'cuisine_count', 'places', 'locality', 'region', 'city', 'latitude',
       'longitude', 'has_online_delivery', 'has_table_booking']
other_cols = [c for c in data.columns if c not in col_order]
data = data[col_order+other_cols]
data.sample(5)

,name,aggregate_rating,rating_text,votes,average_cost_for_two,price_range,cuisines,cuisine_count,places,locality,...,Steak,Street Food,Sushi,Tapas,Tea,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese
727,Billy Boo's Cakery,3.7,Good,26,130,2,"Cafe, Patisserie, Bakery",3,None,Observatory,...,0,0,0,0,0,0,0,0,0,0
629,Klein Roosboom,3.7,Good,27,100,2,"Drinks Only, Finger Food",2,None,Durbanville,...,0,0,0,0,0,0,0,0,0,0
1401,North Wharf Restaurant - Protea Hotel,3.0,Average,4,450,4,"American, African, Italian",3,None,Foreshore,...,0,0,0,0,0,0,0,0,0,0
675,Aandklas,3.4,Average,29,200,3,"Pizza, Beverages",2,None,Stellenbosch,...,0,0,0,0,0,0,0,0,0,0
1633,Julep Cocktail Bar,2.9,Average,4,150,2,"Caribbean, Cuban, Latin American",3,None,CBD,...,0,0,0,0,0,0,0,0,0,0


In [120]:
# Order data by rating desc and export to csv
data.sort_values("aggregate_rating", ascending=False, inplace=False).to_csv('processed/zomato-cape-town-final.csv', 
                                                                            encoding='utf-8-sig', index=False)